In [2]:
from langgraph.graph import StateGraph, MessagesState
from IPython.display import Image, display
import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools import tool
from tavily import TavilyClient
from typing import Literal

In [ ]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")
if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your TAVILY API key: ")
    

In [ ]:
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

@tool
def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """Run a web search"""
    return tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
def call_llm(state: MessagesState):
    messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}

In [ ]:
workflow = StateGraph(MessagesState)
workflow.add_node("llm_node", call_llm)
workflow.set_entry_point("llm_node")
workflow.set_finish_point("llm_node")

checkpointer = MemorySaver()
app = workflow.compile(checkpointer=checkpointer)
config = {"configurable": {"thread_id": "SigmaBoy_2"}}

In [ ]:
# use the compiled app (CompiledStateGraph) which exposes the graph
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
input1 = {"messages": [HumanMessage(content="hello")]}

In [ ]:

results = app.invoke(input1, config)

print("Final result:", results)
ai_msg = results["messages"][-1]
ai_msg.pretty_print()


In [115]:
async for event in app.astream_events(input1, config):
    if event["event"] == "on_chat_model_stream":
        chunk = event["data"]["chunk"].content
        if chunk:
            print(chunk, end="", flush=True)



----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------

Harivansh Rai Bachchan, a name that echoes,
With verses that stir, and spirits that grow.
A poet of life, its struggles and grace,
He painted the world with words, in time and space.

From "Madhushala's" depths, a spirit unfurled,
Of wine and of life, a mystical world.
Where cups were raised high, to joy and to pain,
And wisdom was found, in sunshine and rain.

He spoke of the journey, the path yet unknown,
"Chalo, Man, Mere Chalo," a seed he had sown.
To face every dawn, with courage and might,
And banish the shadows, with inner pure light.

His words were a balm, for hearts that were sore,
A gentle reminder, to always explore.
The depths of the soul, the dreams held within,
And rise from the ashes, to start life again.

He taught us to laugh, when tears tried to fall,
To stand tall

In [116]:
latest_checkpoint = checkpointer.get(config)
messages = latest_checkpoint["channel_values"]["messages"]

last_ai_msg = None
for msg in reversed(messages):
    if msg.__class__.__name__ == "AIMessage":
        last_ai_msg = msg.content
        break

print(last_ai_msg)

Harivansh Rai Bachchan, a name that echoes,
With verses that stir, and spirits that grow.
A poet of life, its struggles and grace,
He painted the world with words, in time and space.

From "Madhushala's" depths, a spirit unfurled,
Of wine and of life, a mystical world.
Where cups were raised high, to joy and to pain,
And wisdom was found, in sunshine and rain.

He spoke of the journey, the path yet unknown,
"Chalo, Man, Mere Chalo," a seed he had sown.
To face every dawn, with courage and might,
And banish the shadows, with inner pure light.

His words were a balm, for hearts that were sore,
A gentle reminder, to always explore.
The depths of the soul, the dreams held within,
And rise from the ashes, to start life again.

He taught us to laugh, when tears tried to fall,
To stand tall and proud, and answer life's call.
With a spirit unbent, and a heart full of fire,
Harivansh Rai Bachchan, our poetic sire.

His legacy lives on, in every line penned,
A beacon of hope, that will never end